In [ ]:
%%HTML
<style>
.container { width:100% }
</style>

# Reinforcement learning


Beim Reinforcement Learning geht es darum, dass sich die KI ihr Modell innerhalb eines Lernprozesses selber entwickelt. Während des Lernprozesses wird mit Hilfe der `reward_function()` eigenständig eine Strategie entwickelt, mit deren Hilfe ein Ziel erreicht werden kann. Ein Ziel zeichnet sich im Algorithmus dadurch aus, dass der Agent eine Belohnung erhält, wenn er den Ziel-Zustand erreicht. Zum Trainieren werden zufällige Aktionen ausgeführt. 

<img src="img/reinforcement_learning.png" alt="Schema Reinforcement Learning" width="600"/>

Die Abbildung oben zeigt, den Zyklus aus Aktionen, die das Environment beeinflussen und den Erkenntnissen, die der Agent dadurch entwickelt. Im nächsten Schritt nutzt er dieser Erkenntnisse aus dem Folgezustand und dem erhaltenen Reward zur Auswahl der nächsten Aktion.

Mithilfe von verschiedenen Verfahren, die im Laufe dieser Arbeit erläutert werden, kann eine Strategie erstellt werden, die den Agenten zum Ziel führt. Diese Strategie (engl. Policy) wird mit $\pi$ bezeichnet.
Mit den bis jetzt definierten States, Aktionen, Rewards und der Umgebung können nun die Strategien für den Lernprozess angewendet werden.

## Implementierung 3
Um die verschiedenen Algorithmen des Reinforcement Learnings wieder auf unser Beispiel anwenden zu können, wird zunächst eine allgemeine Funktion definiert.

`transport_goods()` führt so oft eine Aktion, die von der gewünschten `action_method()` ausgewählt wird, aus, bis die Ware an ihrem Zielort eingetroffen ist. Nach dem Ausführen wird die Darstellung falls gewünscht aktualisiert. Die Konstante `MAX_VISITS_STATE` gibt an, wie oft ein Zustand besucht werden kann, bevor ausgehend von diesem Zustand statt der eigentlich gewünschten Aktion eine zufällige Aktion gestartet wird. Diese Randomisierung ist notwendig, um zu verhindern, dass sich Aktions-Schleifen bilden, in denen der Agent festhängt. Im idealen Fall besucht der Agenten einen State nur einmal (Position zweimal, einmal mit Paket, einmal ohne). Der Wert wird trotzdem höher angesetzt, um keine zu starke Einschränkung zu bieten.

Bei der Funktion handelt es sich auch um eine Funktion der Klasse `Transport`.

In [ ]:
MAX_VISITS_STATE = 10

In [ ]:
def transport_goods(self):
    position_truck, position_goods, position_goal = self.state
    while not self.done:    
        self.steps += 1
        state_num = state_to_state_num(self.state)
        action = self.action_method(state_num)
        if action not in possible_actions[position_truck]:
            action = random.sample(possible_actions[position_truck], 1)[0]
        
        value = self.reward_function(action)
        new_state = self.transition_function(action)
        if self.visited_states.count(new_state) > MAX_VISITS_STATE:
            action = random.sample(possible_actions[position_truck], 1)[0]
            value  = self.reward_function(action)
            new_state = self.transition_function(action)
        
        self.current_value += value
        self.state = new_state
        self.visited_states.append((self.state))
        position_truck, position_goods, position_goal = self.state
        
        if position_goods == position_goal: self.done = True
        if self. visualize:
            action_label.value = 'Action: ' + actions_description[action]
            self.update_canvas()
        if self.stepwise: time.sleep(0.1)
Transport.transport_goods = transport_goods
del transport_goods

Im Folgenden werden alle möglichen Zustände in einer Liste gespeichert, sodass sie nummerierbar und damit eindeutig identifizierbar sind. Die nachfolgenden Funktionen `state_to_state_num()` und `state_num_to_state()` dienen dazu, zwischen den beiden Darstellungsweisen zu wechseln.

In [ ]:
states = []
for row in range (0, 6):
    for col in range (0, 6):
        for position_goods in range (0, 5):
            for position_goal in range (0, 4):
                states.append(((row, col), position_goods, position_goal))

In [ ]:
def state_to_state_num(state):
    return states.index(state)

def state_num_to_state(num):
    return states[num]

Um einen Vergleich zu haben, wie der Transport ohne Reinforcement Learning aussehen würde, wird die folgende Methode genutzt. Die Funktion `random_action()` gibt eine zufällige Aktion zurück. Sie nutzt also keinen Lerneffekt oder Trainingsdaten. Im weiteren Verlauf der Arbeit werden verschiedene Algorithmen zur Auswahl der Aktionen implementiert, die genau dies tun. In der abschließenden Evaluation wird dann zu sehen sein, wie sich die Nutzung dieser Daten auf die Effizienz des Transports auswirkt.

In [ ]:
def random_action(state_num):
    return random.randrange(0, 6, 1)

## Modell basiert vs. Modell frei

Im Reinforcement Learning lassen sich die Probleme und ihre Lösungsalgorithmen in zwei Ansätze aufgliedern. Zum einen gibt es den Modell basierten Ansatz, häufig als modell-based bezeichnet. Die andere Gruppe heißt Modell frei bzw. modell-free.
Verwendet der Agent während dem Lernvorgang Vorhersagen darüber, welche Reaktion zu erwarten ist, so handelt es sich um einen Modell basierten Ansatz. Dies kann den Umfang einer Stichprobe haben, oder sogar die komplette Information über die Verteilung des nächsten Zustands inklusive aller Belohnungen beinhalten. Modell basiert bedeutet jedoch nicht, dass kein Modell der Umgebung implementiert sein darf, oder das Modelle für Maschinelles Lernen vorhanden sein müssen. Typische Beispiele für Modell basierte Algorithmen sind Policy oder Value Iteration die dem Bereich Dynamic Programming angehören. Basiert ein Algorithmus auf Erfahrungswerten, wie zum Beispiel Monte-Carlo und Q-learning, gehören entsprechend der Gruppe der Modell freien Methoden an. Sie verwenden lediglich Proben aus den bereits erlernten Werten, aber keine Vorhersagen über zukünftige Belohnungen. 
(vgl. Russell und Norvig 2010)

## Passives lernen vs. aktives lernen
Reinforcement Learning Methoden lassen sich in zwei weitere Kategorien aufteilen. Beim passiven Lernen steht eine optimale Policy für den Agenten bereits und er muss bewerten, wie gut die ausführbaren Aktionen sind. Muss der Agent noch lernen, was er tun soll, so spricht man vom aktiven Lernen.

Zunächst werfen wir ein Blick auf passives Lernen. Kennt der Agent die optimale Strategie bereits, um ein Problem zu lösen bedeutet das, dass die Policy $\pi$ bekannt ist. Somit führt er für einen Zustand $s$ immer die Aktion $\pi(s)$ aus. Es gilt herauszufinden, wie sinnvoll und nützlich dies ist. Die Funktion die dies ermittelt nennt man Utility-Funktion $U^{\pi}(s)$. Diese Funktion $U$ gilt es zu finden. Algorithmen, die dies ermöglichen sind zum Beispiel Adaptive dynamic programming (ADP) und Temporal difference learning (TD-learning).

Die zweite Kategorie ist die Gruppe der Probleme, bei denen der Agent zunächst eine Strategie für die gewünschte Aktion ermitteln muss. Er kennt die optimale Policy $\pi$ nicht. Er muss also in jedem Zustand selbst entscheiden, was die beste nächste Aktion ist. So entsteht ein Modell, dass er immer weiter verbessert, um in Zukunft größere Belohnungen zu erhalten.
Dabei kann es passieren, dass er annimmt einen optimalen Weg gefunden zu haben und diesen immer wieder geht. Andere vielleicht genauso gute oder sogar bessere Zustände und Optionen werden nicht mehr betrachtet. Dieses Problem bezeichnet man auch als "exploitation vs. exploration". Bei der exploitation (dt. "Ausbeutung") macht der Agent das zuvor beschriebene: er nutzt Wege, von denen er gelernt hat, dass sie gut sind und optimiert sie gegebenenfalls weiter. Exploration hingegen bezeichnet die Erkundung von neuen Zuständen. Versucht der Agent nur neue Zustände zu besuchen und neue Wege zu analysieren, ist dies auch nicht zielführend. Denn was nützt es, wenn man einen guten Weg kennt, aber ihn die nutzt? Der Agent muss also einen Kompromiss zwischen exploitation und exploration eingehen, um sowohl die kurzfristige als auch langfristige Belohnung zu maximieren. Später wird dafür der Wert $\epsilon$ eingeführt werden. In $\epsilon$-Prozent der Fälle geht der Agent dann neue Wege. In allen anderen Fällen versucht er die bekannten Wege zu gehen und zu lernen. Algorithmen mit denen dies möglich ist sind zum Beispiel ADP mit einer zusätzlichen exploration-Funktion oder der gleich vorgestellte Q-learning-Algorithmus.
(vgl. Russell und Norvig 2010)

Bei unserem Speditions-Beispiel handelt es sich gemäß der zuvor erläuterten Theorie um aktives lernen. Denn unser Agent, der LKW, hat von einem Feld ausgehend häufig viele Optionen in welche Richtung er fährt oder ob er Ware abliefert oder einsammelt. Er weiß, welche Aktionen in welchem Zustand möglich sind, kennt aber ohne Lernvorgänge nicht die Aktion, die am schnellsten zum Ziel führt.  

## Monte-Carlo-Methode

Die Monte-Carlo-Methode ist eine Methode zur Ermittlung der zu erwartenden Belohnung beim Reinforcement Learning. Das besondere bei Monte-Carlo ist, dass es sich um einen randomisierten Algorithmus handelt. Als Voraussetzung  werden Erfahrungswerte, also Trainingsdaten bestehend aus einer Folge von Zuständen, Aktionen und Belohnungen benötigt. Die Idee ist, aus diesen Daten den durchschnittlich erreichten Wert nach besuchen eines Zustands $s$ zu ermitteln. Dazu gibt es zwei Ansätze: "Every-Visit" und "First-Visit". Diese sollen mit Hilfe der beiden folgenden Episoden erklärt werden.

<img src="img/mc-episodes.png" alt="Episoden zur Erklärung von First-Visit und Every-Visit" width="800"/>

_Quelle: Eigene Darstellung nach Gupta 2020_

Bei "First-Visit" werden alle Belohnungen, die zwischen dem ersten Besuch des betrachteten Zustands $s$ und dem Endzustand auftreten, aufsummiert. Die so berechneten Werte für alle Episoden werden aufaddiert und durch die Anzahl der betrachteten Episoden geteilt.

Für unsere Beispiel-Episoden bedeutet dies:

Für den Wert von Zustand $A$:
\begin{equation}
\begin{split}
V_1(A) &= 3 + 2 + (-4) + 4 + (-3) = 2 \\\
V_2(A) &= 3 + (-3) = 0 \\\
V(A) &= \frac{V_1(A) + V_2(A)}{2} = \frac{2+0}{2} = 1
\end{split}
\end{equation}

Für den Wert von Zustand $B$:
\begin{equation}
\begin{split}
V_1(B) &= (-4) + 4 + (-3) = -3 \\\
V_2(B) &= (-2) + 3 + (-3) = -2 \\\
V(B) &= \frac{V_1(B) + V_2(B)}{2} = \frac{(-3)+(-2)}{2} = -2,5
\end{split}
\end{equation}

Bei "Every-Visit" wird ein neuer Summations-Term für jedes Auftreten des Zustands $s$ erstellt. Das bedeutet man summiert zunächst alle Belohnung zwischen dem ersten Auftreten und dem Endzustand, addiert dazu die Summe der Belohnungen zwischen dem zweiten Auftreten und dem Endzustand und so weiter bis zur Addition der Summe der Belohnungen zwischen dem letzten Auftreten und dem Endzustand. Dies wird für alle Episoden wiederholt, aufaddiert und durch die Anzahl der Summations-Terme, also die Anzahl des Auftretens des Zustands $s$, dividiert.

Für unsere Beispiel-Episoden bedeutet dies:

Für den Wert von Zustand $A$:
\begin{equation}
\begin{split}
V_1(A) &= (3 + 2 + (-4) + 4 + (-3)) + (2 + (-4) + 4 + (-3)) + (4 + (-3)) = 2 + (-1) + 1 = 2 \\\
V_2(A) &= 3 + (-3) = 0 \\\
V(A) &= \frac{V_1(A) + V_2(A)}{3 + 1} = \frac{2+0}{4} = 0,5
\end{split}
\end{equation}

Für den Wert von Zustand $B$:
\begin{equation}
\begin{split}
V_1(B) &= ((-4) + 4 + (-3)) + (-3) = -3 + (-3) = -6\\\
V_2(B) &= ((-2) + 3 + (-3)) + (-3) = -2 + (-3) = -5 \\\
V(B) &= \frac{V_1(B) + V_2(B)}{2 + 2} = \frac{(-6)+(-5)}{4} = -2,75
\end{split}
\end{equation}

(vgl. Gupta 2020, Sutton 2015)

In der Anwendung von Monte-Carlo wird eine Episode von einem zufälligen Zustand bis zum Endzustand mit zufälligen Aktionen durchgespielt und dabei Zustände, Aktionen und Belohnungen aufgezeichnet. Anschließend werden die Werte der Zustände nach einer der oben erläuterten Methode berechnet. Anschließend wird ein neuer zufälliger Zustand bestimmt und unter Ausnutzung der zuvor berechneten Werte bis zum Ende geführt. Die Werte werden nach dem Durchgang neu berechnet und gemittelt. Dies wird viele Male wiederholt. So ergibt sich die gewünschte Werte-Matrix Q.

Bei diesem Algorithmus ist nicht sichergestellt, dass jeder Zustand nach dem Training besucht wurde. Ein weiterer Nachteil ist, dass das Problem den Endzustand erreichen muss, bevor die Werte berechnet werden können. Es muss sich also zwingend um terminierende Probleme handeln.

## Implementierung 4

Für die Monte-Carlo-Methode wird neben der Werte-Tabelle auch noch eine Tabelle benötigt, die zählt, welche Aktion in welchem Zustand wie oft aufgerufen wurde.

In [ ]:
%run ./02_markov_decision_process.ipynb
%run ./implementierung_gui.ipynb

In [ ]:
mc_table = np.zeros([len(states), len(actions)])
count_state_action = np.zeros([len(states), len(actions)])

Die Funktion `mc_function` passt nach abschließen einer Episode die Werte in der Tabelle an. Dafür wird das eben vorgestellte "First-Visit"-Prinzip genutzt.

In [ ]:
def mc_function(state_list):
    states = [s for (s, _,_) in state_list]
    state_action_pairs = {(s, a) for (s, a, _) in state_list}
    for (state, action) in state_action_pairs:
        idx = states.index(state)
        value = 0
        for i in range (idx, len(state_list)):
            value += state_list[i][2]
        state_num = state_to_state_num(state)
        count_state_action[state_num][action] += 1
        mc_table[state_num][action] = (mc_table[state_num][action] + value)/count_state_action[state_num][action]

`mc_learning` berechnet die Werte-Tabelle mit einer zu übergebenden Anzahl an Episoden-Durchläufen. Dabei wird das Exploitation und Exploration-Prinzip verwendet.

In [ ]:
def best_actions(table, state_num):
    return [i[0] for i in np.argwhere(table[state_num] == np.amax(table[state_num]))]

In [ ]:
def mc_learning(episodes):
    epsilon = 0.3
    
    for episode in tqdm(range(episodes)):
        if episode%100 == 0:
            np.savetxt('mc_table_'+str(episode)+'.txt', mc_table)
        state = states[random.randrange(0, len(states), 1)]
        done  = False
        state_list = []
        transport  = Transport(state, None, stepwise = False, visualize = False)
        
        while not done:
            state_num = state_to_state_num(transport.state)
            if random.uniform(0, 1) < epsilon:
                action = random.sample(possible_actions[transport.state[0]], 1)[0] # Aktionsbereich erkunden
            else:
                b = best_actions(mc_table, state_num)
                action = random.sample(b, 1)[0] # Gelernte Werte ausnutzen
            reward     = transport.reward_function(action)
            transport.state = transport.transition_function(action)
            state_list.append((transport.state, action, reward))
            _, position_goods, position_goal = transport.state
            if position_goods == position_goal: done = True
        mc_function(state_list)

Achtung, die Ausführung des nächsten Befehls dauert länger. 

Zum Trainieren muss die Funktion `mc_learning()` mit einem relativ hohen Wert aufgerufen werden. Damit dies nicht nach jedem Neustart des Kernels notwendig ist, kann die Tabelle abgespeichert und beim nächsten Mal einfach geladen werden.

In [ ]:
mc_learning(1000) # Trainieren
#np.savetxt('mc_table.txt', mc_table) # Speichern der Tabelle
#mc_table = np.loadtxt('mc_table.txt') # Laden der Tabelle

Die folgende Methode kann dann nach dem Trainieren in der tatsächlichen Anwendung genutzt werden, um für jeden Zustand die beste Aktion auf Basis der zuvor berechneten Werte zu ermitteln. Dazu muss sie der Klasse `Transport` bei der Initialisierung übergeben werden und wird später dann von `transport_goods()` aufgerufen.

In [ ]:
def mc_action(state_num):
    return np.argmax(mc_table[state_num]) 

Die Erkenntnis aus dieser Implementierung ist, dass die Monte-Carlo-Methode für das hier vorliegende Problem nicht sehr gut geeignet ist. Der Grund dafür ist, dass gerade die Durchläufe am Anfang sehr lange dauern, da die in der Tabelle gespeicherten Werte für jede Aktion identisch sind. Der Agent nimmt suboptimale Aktionen und versucht diese zu nutzen, gelangt jedoch nicht zum Ziel. Da sich die Werte erst verbessern, wenn die Episode terminiert, kann der Agent innerhalb einer Episode seinen Weg nicht optimieren und wieder schlechte und nicht zielführende Aktionen. Für eine gute Lösung des hier vorliegenden Problems eignen sich entsprechen Algorithmen besser, die eine Optimierung nach jedem Schritt ermöglichen.

## Temporal-difference learning (TD-Learning)

Ähnlich wie bei Monte Carlo, wird Temporal-Difference-Learning zur Berechnung der Q-Matrix verwendet. Bei jedem Durchlauf entscheidet sich der Agent für die Aktion, dessen Folgezustand die höchsten Belohnung bereitstellt. Im Anschluss wird der Wert innerhalb der Q-Matrix mit Hilfe der Value-Funktion nach folgender Formel angepasst:
    
$V(s)\leftarrow V(s)+\alpha(R + \gamma V(s') - V(s))$

wobei gilt:
* $V(s)$ ist der Wert des aktuellen States
* $V(s')$ ist der Wert des nächsten States
* $R$ ist der Reward, der im nächstem Schritt erhalten wird
* $\gamma$ ist die Gewichtung, wie hoch der Einfluss des Wertes in den Gesamtwert eingeht (engl. Discount-Factor) 
* $\alpha$ ist die Lernrate

Der Teil $R + \gamma V(s') - V(s)$ wir auch als TargetError bezeichnet.

Nach jeder Aktion des Agenten wird die Q-Matrix optimiert und $V$ nähert sich $V^{\pi}$ an. Im Gegensatz zu Monte Carlo wird die Matrix also nicht erst angepasst, wenn ein Reward erhalten wurde. Zudem werden die Rewards, mit einem Gewicht $\gamma$ versehen. Dadurch wird aktuelleren Werten eine höhere Wertigkeit im Endergebnis gegeben. Der Vorteil dabei ist, dass die Varianz minimiert wird. Ausreißer haben also kleinere Auswirkungen auf das Endergebnis.

<img src="img/td_learning.JPG" alt="td learning" width="400"/>

Auf der Abbildung ist zu sehen, wie sich der Wert (gestrichelte Linie) nach jeder ausgeführten Aktion optimiert, wobei auf der x-Achse der zeitliche Ablauf und auf der y-Achse der Wert abgebildet ist.


(vgl. Kumar 2019, Willcocks 2021)

## Q-learning

Q-learning ist ein modellfreier Reinforcement Learning Algorithmus, mit dem der Wert einer Aktion berechnet werden kann. Er ist eine Variante von TD-Learning und benötigt keine Anpassung des Environments, lediglich Rewards und stochastische Transitionen. Für einen endlichen Markov-Decision-Prozess ist Q-learning in der Lage eine Policy zu erstellen. Eine optimale Policy $\pi*$ auf Basis von zufälligen Aktionen zu erstellen, wäre mit Q-Learning möglich, falls unendlich viel Zeit zur Verfügung stünde.

In eine Matrix der Form $len(States) \times len(Aktionen)$ werden die Rewards eingetragen, die für eine Aktion erhalten werden, die in einem State ausgeführt wird. Diese Matrix wird im Code als `q_table` bezeichnet und ist initial mit Nullen oder niedrigen zufälligen Werten gefüllt.  Die Q-Funktion: $Q: S x A \rightarrow \mathbb{R}$ mappt eine Aktion und einen State auf eine rationale Zahl, die in der Matrix an der Position `q_table[state][action]` eingetragen wird. Neue Werte werden mit einer höheren Gewichtung in die Matrix einberechnet. Die Gewichtung gibt der Discount-Faktor $\gamma$ an. In den Werten der Matrix sind die Werte der letzten States aufsummiert eingetragen, also die Rewards des ganzen Weges.

## Implementierung 5

Zunächst wird eine Tabelle mit einer Zeile für jeden Zustand und einer Spalte für jede Aktion angelegt. In jede Zelle wird zu Beginn der Wert Null geschrieben.

In [ ]:
q_table = np.zeros([len(states), len(actions)])

Die `q_function()` nimmt einen Zustand und eine Aktion sowie die Lernrate $\alpha$ und den Discount-Faktor $\gamma$. Darauf basierend berechnet sie mit Hilfe der `q_table` den neuen Wert für den Zustand und die Aktion und trägt diesen an der entsprechenden Stelle in der Tabelle ein. Zurückgegeben wird außerdem der nächste Zustand. Die Formel für die Berechnung des neuen Wertes lautet folgendermaßen:

\\[Q_{\texttt{new}}(s_t, a_t) \leftarrow Q_{\texttt{old}}(s_t, a_t) + \alpha \cdot (r_t + \gamma \cdot max_q(s{t+1}, a) - Q_{\texttt{old}}(s_t, a_t))\\]

mit:
* $s_t$, $s_{t+1} \in$ len(states)
* a $\in$ actions
* $Q_{\texttt{old}}(s_t, a_t)$ = Tabellenwert in Zeile $s_t$ und Spalte $a_t$
* $\alpha$ = Lernrate
* $\gamma$ = Discount-Faktor
* $r_t$ = Reward
* max_q = Schätzung des optimalen Zukunftswertes

Der Wert in der Klammer wird dabei auch als "temporal difference" bezeichnet.

In [ ]:
def q_function(state, action, alpha, gamma):
    transport  = Transport(state, stepwise = False, visualize = False)
    
    reward     = transport.reward_function(action)
    next_state = transport.transition_function(action)
    
    state_num      = state_to_state_num(state)
    next_state_num = state_to_state_num(next_state)
    
    old_value  = q_table[state_num][action]
    max_q      = np.max(q_table[next_state_num])
    new_value  = old_value + alpha * (reward + gamma * max_q - old_value)
    
    q_table[state_num, action] = new_value
    return next_state

Die Funktion `q_learning()` führt für eine zu übergebende Anzahl an Startzuständen (`episodes`) den Transport der Waren zum Ziel durch. Der Startzustand ist dabei ein zufälliger. Dies dient dem Training und somit der Verbesserung der `q_table`. Denn in jedem Schritt wird `q_function()` aufgerufen.  In rund 10% (= `epsilon`) der Schritte ist auch die Aktion zufällig ausgewählt. In den anderen Fällen wird die am besten bewertete Aktion für den aktuellen Zustand genutzt.

In [ ]:
def q_learning(episodes):
    alpha   = 0.1
    gamma   = 0.6
    epsilon = 0.1
    
    for episode in tqdm(range(episodes)):
        state = states[random.randrange(0, len(states), 1)]
        done  = False
        
        while not done:
            state_num = state_to_state_num(state)
            if random.uniform(0, 1) < epsilon:
                action = random.sample(possible_actions[state[0]], 1)[0] # Aktionsbereich erkunden
            else:
                action = np.argmax(q_table[state_num]) # Gelernte Werte ausnutzen
                
            state = q_function(state, action, alpha, gamma)
            _, position_goods, position_goal = state
            if position_goods == position_goal: done = True

Zum Trainieren muss die Funktion `q_learning()` mit einem relativ hohen Wert aufgerufen werden. Damit dies nicht nach jedem Neustart des Kernels notwendig ist, kann die Tabelle abgespeichert und beim nächsten Mal einfach geladen werden.

In [ ]:
#q_learning(10000) # Trainieren
#np.savetxt('q_table.txt', q_table) # Speichern der Tabelle
q_table = np.loadtxt('q_table.txt') # Laden der Tabelle

Die folgende Methode kann dann nach dem Trainieren in der tatsächlichen Anwendung genutzt werden, um für jeden Zustand die beste Aktion zu ermitteln. Dazu muss sie der Klasse `Transport` bei der Initialisierung übergeben werden und wird später dann von `transport_goods()` aufgerufen.

In [ ]:
def q_learning_action(state_num):
    return np.argmax(q_table[state_num]) 